In [ ]:
# ideally:
# predict radius, center on BxCxHxW
# perform outlier rejection on radius/center

# currently:
# predict radius, center on reduced TxCxHxW segmentation

import sys
sys.path.append("/home/martin/Dev/homography_imitation_learning")
import os
from scipy import stats
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 300

from utils.io import recursive_scan2df

prefix = "/media/martin/Samsung_T5/data/endoscopic_data/cholec80_circle_tracking_individual"
data_df = recursive_scan2df(prefix, ".pkl")
# data_df = data_df.sort_values(["folder", "file"]).reset_index(drop=True)

outlier_rejection = False


for idx, row in data_df.iterrows():
    print(row.folder, row.file)
    df = pd.read_pickle(os.path.join(prefix, row.folder, row.file))
    
    df.center = df.center.apply(lambda x: np.array(x))

    if outlier_rejection:
        try:
            # radius
            z_score = np.abs(stats.zscore(df.radius, nan_policy='omit'))
            df.loc[z_score > 3, 'radius'] = np.nan
            df.radius = df.radius.interpolate(method='pad')
            # df.radius = df.radius.rolling(window=25).mean()

            # center
            df.center = df.center.apply(lambda x: x if x is not np.nan else np.full(2, np.nan))
            z_score = stats.zscore(np.stack(df.center.to_numpy()), nan_policy='omit')
            z_score = np.linalg.norm(z_score, axis=1)
            df.loc[z_score > 3, 'center'] = np.nan
            df.center = df.center.apply(lambda x: x if x is not np.nan else np.full(2, np.nan))
            df.center = df.center.interpolate(method='pad')
        except:
            continue

        plt.plot(df.frame, df.radius, label="Video {}".format(idx))
        plt.grid()
        plt.legend()
        plt.xlabel("Frame / #")
        plt.ylabel("Radius / pixels")
        plt.savefig(prefix + "/figs_interpolated/radius/radius_video_{}.png".format(idx), transparent=False)
        plt.clf()

        plt.scatter(np.stack(df.center.to_numpy())[:,0], np.stack(df.center.to_numpy())[:,1], label="Video {}".format(idx))
        plt.grid()
        plt.legend()
        plt.savefig(prefix + "/figs_interpolated/center/center_video_{}.png".format(idx), transparent=False)
        plt.clf()

    else:
        plt.plot(df.frame, df.radius, label="Video {}".format(idx))
        plt.grid()
        plt.legend()
        plt.xlabel("Frame / #")
        plt.ylabel("Radius / pixels")
        plt.savefig(prefix + "/figs/radius/radius_video_{}.png".format(idx), transparent=False)
        plt.clf()


        df.center = df.center.apply(lambda x: x if x is not np.nan else np.full(2, np.nan))
        plt.scatter(np.stack(df.center.to_numpy())[:,0], np.stack(df.center.to_numpy())[:,1], label="Video {}".format(idx))
        plt.grid()
        plt.legend()
        plt.savefig(prefix + "/figs/center/center_video_{}.png".format(idx), transparent=False)
        plt.clf()

